### Stats Exercises week 4

## Problem 1.

(Computer exercise) We will consider a simulation to study null-hypothesis significance testing (NHST).

The researchers are using a test with level $\alpha = 0.05$.

In the following cases, compute the expected number of:

true positive rejections (TP) where $H_1$ is true and $H_0$ is correctly rejected, 

false positives (FP) where $H_0$ is true but it is rejected, 

true negatives (TN) where $H_0$ is true and it is not rejected, 
 
and false negatives (FN) $H_1$ is true but $H_0$ is not rejected.

In [107]:
import math
def meaningful(x, digits = 3):
    if x == 0:
        return "0"
    magnitude = math.floor(math.log10(abs(x)))
    decimals = digits - magnitude - 1
    add = " "
    if x < 0:
        add = ""
    return add + f"{round(x, decimals):g}"

def calculate_values(h_0, h_1, alpha, power):
    tn = int(round((1 - alpha) * h_0))
    fp = int(round(alpha * h_0))
    tp = int(round(power * h_1))
    fn = int(round((1 - power) * h_1))
    assert tn + fp + tp + fn == h_0 + h_1
    print([tp, fp, tn, fn])
    return f"Test:\n\tH_0: {h_0}\tH_1: {h_1}\tpow: {power}\tpow: {alpha}\n\ttp: {tp}\t\tfp: {fp}\t\ttn: {tn}\t\tfn: {fn}\n"
    

print(calculate_values(100,100,0.05,0.8))
print(calculate_values(100,100,0.05,0.2))
print(calculate_values(900,100,0.05,0.8))
print(calculate_values(900,100,0.05,0.2))

[80, 5, 95, 20]
Test:
	H_0: 100	H_1: 100	pow: 0.8	pow: 0.05
	tp: 80		fp: 5		tn: 95		fn: 20

[20, 5, 95, 80]
Test:
	H_0: 100	H_1: 100	pow: 0.2	pow: 0.05
	tp: 20		fp: 5		tn: 95		fn: 80

[80, 45, 855, 20]
Test:
	H_0: 900	H_1: 100	pow: 0.8	pow: 0.05
	tp: 80		fp: 45		tn: 855		fn: 20

[20, 45, 855, 80]
Test:
	H_0: 900	H_1: 100	pow: 0.2	pow: 0.05
	tp: 20		fp: 45		tn: 855		fn: 80



## Problem 2.

A scientific paper in machine learning reports a comparison of three algorithms on a classification task (average accuracy $\pm$ standard deviation over 8 repeats with an independent random partition of the data; higher is better).

| Algorithm    | Accuracy (%) ± standard deviation (%) |
|--------------|---------------------------------------|
| **Algorithm 1** | 96.0 ± 2.0                           |
| Algorithm 2  | 95.0 ± 2.0                           |
| Algorithm 3  | 93.5 ± 2.0                           |

The authors highlight Algorithm 1 as the most accurate method because it has the highest average accuracy.

Let us study this formally by using the Wald test for differences in average accuracies between each pair of algorithms with the null hypothesis that the average accuracies of the two are the same, using 
$95%$ significance level.

Perform the test for differences of all pairs of algorithms. Report the resulting p-values. Consider if authors’ claim of Algorithm 1 being the best can be accepted.

## Solution

Assume that $\hat \theta$ is asymptomatically normal
 
\begin{aligned}
\frac{\hat \theta - \theta_0}{\hat{se}} & \rightsquigarrow N(0,1)\\
\end{aligned}

We test null hypothesis $p_k = p_l$, where $p_i = P(A_i) = Binomial(n,p_i)$ and $l, k\in 1, 2, 3$

We can write $H_0: \delta = p_k - p_l = 0$

The $MLE$ is $\hat \delta = \hat p_l - \hat p_k$

Its standard error is:
$$
\hat se = \sqrt{V(\hat p_l - \hat p_k)} = \sqrt{V(\hat p_l) + V(\hat p_k)} = \sqrt{\frac{\sigma_l^2}{n} + \frac{\sigma_k^2}{n}}  = \sqrt{ \frac{0.02^2}{8} + \frac{0.02^2}{8} } = 0.01
$$

The test statistic is:

$$
W = \frac{\hat \delta - 0}{\hat se} = \frac{\hat p_l - \hat p_k}{0.01} = 100 \cdot( \hat p_l - \hat p_k)
$$

For test of size $\alpha = 0.05$, we reject when $|W|>z_{\alpha / 2}$

The p-value is $p=2\Phi(-|W|)$

Since test set was partitioned independently, we can assume that the test sets were independent

In [108]:
from math import sqrt
from scipy.stats import norm

algorithm_w = lambda x,y: (x - y)*100
def calculate_p_value(w):
    return 2*norm.cdf(-abs(w))

algorithm_means = [0.96, 0.95, 0.935]
pairs = [(0,1),(0,2),(1, 2)]
for a, b in pairs:
    p_1, p_2 = algorithm_means[a], algorithm_means[b]
    w = algorithm_w(p_1, p_2)
    p_val = calculate_p_value(w)
    print(f"Comparing algorith {a + 1} to {b + 1}")
    print(f"\tW: {meaningful(w)}")
    print(f"\tp: {meaningful(p_val)}")
    print(f"\tnull-hypothesis rejected: {p_val < 0.05}")

Comparing algorith 1 to 2
	W:  1
	p:  0.317
	null-hypothesis rejected: False
Comparing algorith 1 to 3
	W:  2.5
	p:  0.0124
	null-hypothesis rejected: True
Comparing algorith 2 to 3
	W:  1.5
	p:  0.134
	null-hypothesis rejected: False


## Problem 3.

A randomized, double-blind experiment was conducted to assess the effectiveness of several drugs for reducing postoperative nausea. The data are as follows.

| Drug                    | Number of Patients | Incidence of Nausea |
|-------------------------|-------------------:|--------------------:|
| Placebo                 |                 80 |                  45 |
| Chlorpromazine          |                 75 |                  26 |
| Dimenhydrinate          |                 85 |                  52 |
| Pentobarbital (100 mg)  |                 67 |                  35 |
| Pentobarbital (150 mg)  |                 85 |                  37 |


### 3.1.

Test each drug versus the placebo by comparing the rates of incidences of nausea with Wald test. Report the p-values.

The null hypothesis is that the placebo has the same or better effectiveness than a drug.

$$
H_0 : \theta \leq \theta_0
$$

We can write $H_0: \delta = p_p - p_d \leq 0$, where p stands for placebo and d for drug. This means that probability of nausea is smaller for placebo than the drug.

The $MLE$ is $\hat \delta = \hat p_p - \hat p_d$

Its standard error is:
$$
\hat se = \sqrt{V(\hat p_p - \hat p_d)} = \sqrt{V(\hat p_p) + V(\hat p_d)} = \sqrt{\frac{\hat p_p(1- \hat p_p)}{n} + \frac{\hat p_d(1- \hat p_d)}{m}  }
$$

The test statistic is:

$$
W = \frac{\hat \delta - 0}{\hat se} = \frac{\hat p_p - \hat p_d}{\sqrt{\frac{\hat p_p(1- \hat p_p)}{n} + \frac{\hat p_d(1- \hat p_d)}{m}  }} 
$$

A $W>0$ value would indicate that
\begin{aligned}
\hat p_p - \hat p_d &> 0 \\
\hat p_p &> \hat p_d \\
\end{aligned}

Which is the opposite of the null hypothesis. So we can calculate p-value as:

$$
\text{p-value} = P_{\theta_0}(W > w) \approx P(Z > w) = 1 - P(Z \leq w) = 1 - \phi(w)
$$
A $W<0$ value would indicate that
\begin{aligned}
\hat p_p - \hat p_d &< 0 \\
\hat p_p &< \hat p_d \\
\end{aligned}

Which is the opposite of the null hypothesis. So we can calculate p-value as:

$$
\text{p-value} = P_{\theta_0}(W < w) \approx P(Z < w) = \phi(w)
$$


In [110]:
def calculate_W(n, m, p_1, p_2):
    p_1 = p_1 / n
    p_2 = p_2 / m
    denom = p_1 - p_2
    a = p_1*(1-p_1) / n
    b = p_2*(1-p_2) / m
    return denom / sqrt(a + b)


def calculate_p_value_one_sided(w):
    if w >= 0:
        return 1 - norm.cdf(w)
    return norm.cdf(w)

placebo_total = 80
placebo_nause = 45

drugs = [[75, 26, "Chlorpromazine        "], [85, 52, "Dimenhydrinate        "], [67, 35, "Pentobarbital (100 mg)"], [85, 37, "Pentobarbital (150 mg)"]]
p_vals = []
for drug_total, durg_nausea, name in drugs:
    w = calculate_W(placebo_total, drug_total, placebo_nause, durg_nausea)
    p = calculate_p_value_one_sided(w)
    p_vals.append(p)
    print(f"Drug: {name}\tw: {meaningful(w)}\tp_val: {meaningful(p)}")

Drug: Chlorpromazine        	w:  2.76	p_val:  0.00285
Drug: Dimenhydrinate        	w: -0.643	p_val:  0.26
Drug: Pentobarbital (100 mg)	w:  0.486	p_val:  0.313
Drug: Pentobarbital (150 mg)	w:  1.65	p_val:  0.0498



### 3.2.
Report the estimated odds-ratios.

In [111]:
drugs_odds = [
    ["Placebo               ", 80, 45],
    ["Chlorpromazine        ", 75, 26],
    ["Dimenhydrinate        ", 85, 52],
    ["Pentobarbital (100 mg)", 67, 35],
    ["Pentobarbital (150 mg)", 85, 37]
    ]

placebo = 45 / 80
for name, drug_total, durg_nausea in drugs_odds[1:]:
    print(f"Odds-ratio for {name}: {meaningful(placebo/(durg_nausea / drug_total))}\t{meaningful(placebo)}, {meaningful(durg_nausea/drug_total)}")

Odds-ratio for Chlorpromazine        :  1.62	 0.562,  0.347
Odds-ratio for Dimenhydrinate        :  0.919	 0.562,  0.612
Odds-ratio for Pentobarbital (100 mg):  1.08	 0.562,  0.522
Odds-ratio for Pentobarbital (150 mg):  1.29	 0.562,  0.435


## Problem 4.

Continuing with the same data and setting as in the previous exercise.

Use the Bonferroni and the FDR method to adjust for multiple testing. Report the adjusted p-values. Consider how the adjustment affects the conclusions of the test when using 95% significance level.

Since there are four tests, we can simply adjust the p-values by multiplying them by $4$

In [112]:
for i in range(4):
    print(f"Bonferonni: {drugs_odds[i+1][0]}:\t{meaningful(min([p_vals[i] * 4, 1]))}")

Bonferonni: Chlorpromazine        :	 0.0114
Bonferonni: Dimenhydrinate        :	 1
Bonferonni: Pentobarbital (100 mg):	 1
Bonferonni: Pentobarbital (150 mg):	 0.199


FDR Method:

In [113]:
from scipy.stats import false_discovery_control

adjusted = false_discovery_control(p_vals)
for i in range(4):
    print(f"FDR: {drugs_odds[i+1][0]}:\t{meaningful(adjusted[i])}")

FDR: Chlorpromazine        :	 0.0114
FDR: Dimenhydrinate        :	 0.313
FDR: Pentobarbital (100 mg):	 0.313
FDR: Pentobarbital (150 mg):	 0.0996


## Problem 5.

In this exercise, we will compare Bayesian and maximum likelihood estimation for the multinomial distribution.

Multinomial distribution is a generalisation of the binomial to $k$ values with vector parameter $p=(p_1,...,p_k),\sum_{i=1}^k p_i, p_1 \geq 0, i = 1,...,k$ denoting the probabilities of the different outcomes.

Consider $n=20$ throws $X_1,...,X_n$ from an 6-sided die, modelled as 20 draws from a multinomial distribution with unknown $p$. To test the different approaches, we consider a sequence simulated from a fair 6-sided die (the observations are integers between 1 and 6)

$(6,3,2,1,6,6,2,6,1,3,3,2,2,2,5,3,1,6,5,1)$

## 5.1.

Compute the maximum likelihood estimate of $p, \hat p_j = \frac{1}{n}\sum_{i=1}^n I(X_i = j)$

We can get the maximum likelihoods for $p=(\hat p_1,...,\hat p_n)$ by calculating the proportions of results in the sample.

In [114]:
sample = [6,3,2,1,6,6,2,6,1,3,3,2,2,2,5,3,1,6,5,1]
probs = {i+1:sample.count(i+1) / len(sample) for i in range(6)}
for key, val in probs.items():
    print(f"p_{key}: {val}")


p_1: 0.2
p_2: 0.25
p_3: 0.2
p_4: 0.0
p_5: 0.1
p_6: 0.25


## 5.2

Using the $\hat p$, compute the predictive likelihood (i.e. likelihood of future observations) of observing the following sequence in the future: $(1, 5, 4)$


### Answer:

If likelihood of future observations is $\prod_{i=1}^k I(X_i = j)$ and $I(X_3 = 4) = 0$, then the likelihood is also $0$. (Even one 0 term in the product means the whole product is 0)

## 5.3.


For Bayesian inference, we use the conjugate prior $ p \sim \text{Dirichlet}(\alpha) $, where $ \alpha = (\alpha_1, \ldots, \alpha_k), \, \alpha_i > 0, \, i = 1, \ldots, k $.

We assume that all values are equally likely *a priori* (before seeing the data), which implies $ \alpha_1 = \cdots = \alpha_k $.

The Dirichlet distribution is a distribution over probability vectors. Given  
$ p \sim \text{Dirichlet}(\alpha) $, the expectation is:  
$$
\mathbb{E}(p_i) = \frac{\alpha_i}{\alpha_0}, \quad \text{where } \alpha_0 = \sum_{j=1}^k \alpha_j
$$  
The marginal distribution of $ p_i $ is $ \text{Beta}(\alpha_i, \alpha_0 - \alpha_i) $.


Given an observed sequence $ X_1, \ldots, X_n $, the posterior of $ p $ is:

$$
p \mid X_1, \ldots, X_n \sim \text{Dirichlet}(\alpha^*)
$$

with  
$$
\alpha^* = (\alpha_1^*, \ldots, \alpha_k^*), \quad \alpha_j^* = \alpha_j + \sum_{i=1}^n \mathbf{I}(X_i = j)
$$


**Compute the posterior distribution** given the following prior distributions:

1. $ \alpha_1 = \cdots = \alpha_k = 0.1 $  
2. $ \alpha_1 = \cdots = \alpha_k = 1 $  
3. $ \alpha_1 = \cdots = \alpha_k = 10 $

**What do you observe? What kind of prior should we use in a case where we are unsure if the die is fair or loaded?**

### 5.3.1 $ \alpha_1 = \cdots = \alpha_k = 0.1 $  

$$\alpha_j^* = \alpha_j + \sum_{i=1}^n \mathbf{I}(X_i = j)$$

$$\mathbb{E}(p_l) = \frac{\alpha_l}{\alpha_0}, \quad \text{where } \alpha_0 = \sum_{i=1}^k \alpha_i$$

Therefore:

$$
\mathbb{E}(p_l^*)= \frac{\alpha_j + \sum_{i=1}^n \mathbf{I}(X_i = j)}{\alpha_0}
$$


In [116]:
counts = {i+1:sample.count(i+1)for i in range(6)}

def calculate_posterior(a):
    new_alphas = []
    for _, value in counts.items():
        new_alphas.append(a + value)

    adjusted_values = []
    for new_a in new_alphas:
        adjusted_values.append(new_a / sum(new_alphas) )
    info = f"a: {a}\n\t"
    for i in range(1, 7):
        info += f"p_{i}: {meaningful(adjusted_values[i-1], 3)}, "
    print(info)

calculate_posterior(0.1)
calculate_posterior(1)
calculate_posterior(10)


a: 0.1
	p_1:  0.199, p_2:  0.248, p_3:  0.199, p_4:  0.00485, p_5:  0.102, p_6:  0.248, 
a: 1
	p_1:  0.192, p_2:  0.231, p_3:  0.192, p_4:  0.0385, p_5:  0.115, p_6:  0.231, 
a: 10
	p_1:  0.175, p_2:  0.188, p_3:  0.175, p_4:  0.125, p_5:  0.15, p_6:  0.188, 
